In [1]:
# Imports
import ffmpeg 

I was experiencing problems with calling ffmpeg. Not sure exactly what caused it, but after finding a way to display the errors from ffmpeg, it turned out that it was looking for a library that was not available. According to https://stackoverflow.com/questions/62213783/ffmpeg-error-while-loading-shared-libraries-libopenh264-so-5  this is because of a mismatch between python version and ffmpeg version...? Anyway, calling "which ffmpeg" in base and massimal environments showed that two different versions were being used. Trying to use ffmpeg from the command line worked in base, but not in massimal. After uninstalling the version in massimal (conda remove ffmpeg), the correct version (in usr/bin/ffmpeg) is used. However, finding out how to use the ffmpeg-python bindings is still a challenge.

See https://stackoverflow.com/questions/27568254/how-to-extract-1-screenshot-for-a-video-with-ffmpeg-at-a-given-time for tips on extracting a single frame at a given time.

Also see https://developpaper.com/arbitrary-extraction-of-video-frames-by-ffmpeg-python/ for use of python-ffmpeg for this purpose.

-ss   Position option (before -i)

In [26]:
# Paths
video_file = '/media/mha114/MassimalDataProcessing/20220324_VideoTransectTest/GX010199.MP4'
#output_image = '/media/mha114/MassimalDataProcessing/20220324_VideoTransectTest/test_im.jpg'
output_image = '/media/mha114/MassimalDataProcessing/20220324_VideoTransectTest/test_im4.jpg'
pos_file = '/media/mha114/MassimalDataProcessing/20220324_VideoTransectTest/2022-03-24_144702_Bodo/2022-03-24_144702_Bodo.csv'

In [18]:
probe = ffmpeg.probe(video_file)
time = float(probe['streams'][0]['duration'])

In [19]:
time

213.479933

In [13]:
# This works
try:
    out, _ = (
        ffmpeg
        .input(video_file,ss='00:02:09.235')
        .output(output_image, vframes=1, format='image2', vcodec='mjpeg')
        .run(capture_stdout=True, capture_stderr=True, overwrite_output=True)
    )
except ffmpeg.Error as e:
    print(e.stderr)

In [20]:
# A version of the code above, as a function. Time probably needs to be specified as a string
def read_frame_by_time(in_file, time):
    """
    Specify the time node to read any frame
    """
    out, err = (
        ffmpeg.input(in_file, ss=time)
              .output('pipe:', vframes=1, format='image2', vcodec='mjpeg')
              .run(capture_stdout=True)
    )
    return out

In [27]:
# Trying a version with time only given in seconds - also works
try:
    out, _ = (
        ffmpeg
        .input(video_file,ss='129.235')
        .output(output_image, vframes=1, format='image2', vcodec='mjpeg', **{'q:v': 3})  # q:v parameter indicates (photo) quality. Can be omitted. Higher number = worse quality.
        .run(capture_stdout=True, capture_stderr=True, overwrite_output=True)
    )
except ffmpeg.Error as e:
    print(e.stderr)

In [21]:
# This returns the image as a set of bytes
# test_im = read_frame_by_time(video_file,'00:00:20.000')
test_im = read_frame_by_time(video_file,20)

ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

In [23]:
type(test_im)

bytes

In [3]:
# This also works, but is less logical/structured (to me, at least)
stream = ffmpeg.input(video_file, ss='00:00:12.235')
stream = ffmpeg.output(stream, output_image, vframes=1)
ffmpeg.run(stream)

ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

(None, None)